In [1]:
import os
os.chdir('C:/Satyaki/hackathons/e_commerce_gender_prediction')
os.getcwd()

'C:\\Satyaki\\hackathons\\e_commerce_gender_prediction'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,KFold,cross_val_score,RandomizedSearchCV
import joblib
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_data=pd.read_csv('train_8wry4cB.csv',na_values=[' ','?','*'],index_col='session_id')
test_data=pd.read_csv('test_Yix80N0.csv',na_values=[' ','?','*'],index_col='session_id')

In [4]:
train_data.head()

,startTime,endTime,ProductList,gender
session_id,,,,
u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,female
u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,male
u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,female
u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,female
u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,male


In [5]:
train_data.shape

(10500, 4)

In [6]:
train_data.isnull().sum()

startTime      0
endTime        0
ProductList    0
gender         0
dtype: int64

In [7]:
test_data.shape

(4500, 3)

In [8]:
train_data['ProductList'].str.split(';',expand=True)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
session_id,,,,,,,,,,,,,,,,,,,,,
u16159,A00002/B00003/C00006/D28435/,A00002/B00003/C00006/D02554/,A00002/B00003/C00006/D28436/,A00002/B00003/C00006/D28437/,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u10253,A00001/B00009/C00031/D29404/,A00001/B00009/C00031/D02617/,A00001/B00009/C00031/D29407/,A00001/B00009/C00031/D29410/,A00001/B00009/C00031/D29411/,A00001/B00009/C00031/D25444/,A00001/B00009/C00031/D29418/,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u19037,A00002/B00001/C00020/D16944/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u14556,A00002/B00004/C00018/D10284/,A00002/B00004/C00018/D10285/,A00002/B00004/C00018/D10286/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u24295,A00001/B00001/C00012/D30805/,A00001/B00001/C00012/D30806/,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
u15442,A00002/B00002/C00007/D06407/,A00002/B00002/C00007/D06409/,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u17986,A00006/B00030/C00334/D11660/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u22508,A00002/B00002/C00007/D18028/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [9]:
test_data['ProductList'].str.split(';',expand=True)

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
session_id,,,,,,,,,,,,,,,,,,,,,
u12112,A00002/B00003/C00006/D19956/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u19725,A00002/B00005/C00067/D02026/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u11795,A00002/B00002/C00004/D12538/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u22639,A00002/B00003/C00079/D22781/,A00002/B00003/C00079/D22782/,A00002/B00003/C00079/D19325/,A00002/B00003/C00079/D22786/,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u18034,A00002/B00001/C00010/D23419/,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
u23966,A00001/B00031/C00044/D03435/,A00001/B00031/C00044/D01860/,A00001/B00031/C00044/D01690/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u20527,A00002/B00002/C00004/D17474/,A00002/B00002/C00002/D17477/,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u13253,A00002/B00007/C00098/D03350/,A00002/B00007/C00098/D03351/,A00002/B00007/C00098/D03352/,A00002/B00007/C00041/D03354/,A00002/B00007/C00041/D03356/,A00002/B00007/C00041/D03357/,A00002/B00007/C00041/D01940/,A00002/B00007/C00041/D03362/,A00002/B00007/C00041/D03364/,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
total_data=[train_data,test_data]

#### Product list contains list of products viewed by the user in the given session and it also contains the category, sub category, sub-sub category and the product all encoded and separated with a slash symbol. Each consecutive product is separated with a semicolon.

In [11]:
product_list1=['Product_'+str(w) for w in range(36)]
product_list2=['Product_'+str(w) for w in range(43)]

train_data[product_list1]=train_data['ProductList'].str.split(';',expand=True)
train_data.drop('ProductList',axis=1,inplace=True)

test_data[product_list2]=test_data['ProductList'].str.split(';',expand=True)
test_data.drop('ProductList',axis=1,inplace=True)

In [12]:
train_data.head()

,startTime,endTime,gender,Product_0,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,...,Product_26,Product_27,Product_28,Product_29,Product_30,Product_31,Product_32,Product_33,Product_34,Product_35
session_id,,,,,,,,,,,,,,,,,,,,,
u16159,15/12/14 18:11,15/12/14 18:12,female,A00002/B00003/C00006/D28435/,A00002/B00003/C00006/D02554/,A00002/B00003/C00006/D28436/,A00002/B00003/C00006/D28437/,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u10253,16/12/14 14:35,16/12/14 14:41,male,A00001/B00009/C00031/D29404/,A00001/B00009/C00031/D02617/,A00001/B00009/C00031/D29407/,A00001/B00009/C00031/D29410/,A00001/B00009/C00031/D29411/,A00001/B00009/C00031/D25444/,A00001/B00009/C00031/D29418/,...,None,None,None,None,None,None,None,None,None,None
u19037,01/12/14 15:58,01/12/14 15:58,female,A00002/B00001/C00020/D16944/,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u14556,23/11/14 2:57,23/11/14 3:00,female,A00002/B00004/C00018/D10284/,A00002/B00004/C00018/D10285/,A00002/B00004/C00018/D10286/,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u24295,17/12/14 16:44,17/12/14 16:46,male,A00001/B00001/C00012/D30805/,A00001/B00001/C00012/D30806/,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [13]:
test_data.head()

,startTime,endTime,Product_0,Product_1,Product_2,Product_3,Product_4,Product_5,Product_6,Product_7,...,Product_33,Product_34,Product_35,Product_36,Product_37,Product_38,Product_39,Product_40,Product_41,Product_42
session_id,,,,,,,,,,,,,,,,,,,,,
u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/,A00002/B00003/C00079/D22782/,A00002/B00003/C00079/D19325/,A00002/B00003/C00079/D22786/,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## Preprocessing

In [14]:
# separating out product categories and sub-categories into different columns

for i in product_list1:
    train_data[i+'_category']=train_data[i].str.split('/',expand=True).loc[:,0]
    train_data[i+'_sub_category']=train_data[i].str.split('/',expand=True).loc[:,1]
    train_data[i+'_sub_sub_category']=train_data[i].str.split('/',expand=True).loc[:,2]
train_data.drop(product_list1,axis=1,inplace=True)  

for i in product_list2:
    test_data[i+'_category']=test_data[i].str.split('/',expand=True).loc[:,0]
    test_data[i+'_sub_category']=test_data[i].str.split('/',expand=True).loc[:,1]
    test_data[i+'_sub_sub_category']=test_data[i].str.split('/',expand=True).loc[:,2]
test_data.drop(product_list2,axis=1,inplace=True)  

In [15]:
# making train and test data of similar dimensions
train_columns=train_data.columns
test_columns=test_data.columns
missing_columns=test_columns.difference(train_columns)
test_data.drop(missing_columns,axis=1,inplace=True)

In [16]:
# calculating duration from the time columns
for data in total_data:
    data['startTime']=data['startTime'].apply(lambda x: datetime.strptime(x,'%d/%m/%y %H:%M').timestamp())
    data['endTime']=data['endTime'].apply(lambda x: datetime.strptime(x,'%d/%m/%y %H:%M').timestamp())
    data['duration']=data['endTime']-data['startTime']
    data.fillna('**',inplace=True)

In [17]:
train_data.head()

,startTime,endTime,gender,Product_0_category,Product_0_sub_category,Product_0_sub_sub_category,Product_1_category,Product_1_sub_category,Product_1_sub_sub_category,Product_2_category,...,Product_33_category,Product_33_sub_category,Product_33_sub_sub_category,Product_34_category,Product_34_sub_category,Product_34_sub_sub_category,Product_35_category,Product_35_sub_category,Product_35_sub_sub_category,duration
session_id,,,,,,,,,,,,,,,,,,,,,
u16159,1.418647e+09,1.418647e+09,female,A00002,B00003,C00006,A00002,B00003,C00006,A00002,...,**,**,**,**,**,**,**,**,**,60.0
u10253,1.418721e+09,1.418721e+09,male,A00001,B00009,C00031,A00001,B00009,C00031,A00001,...,**,**,**,**,**,**,**,**,**,360.0
u19037,1.417430e+09,1.417430e+09,female,A00002,B00001,C00020,**,**,**,**,...,**,**,**,**,**,**,**,**,**,0.0
u14556,1.416692e+09,1.416692e+09,female,A00002,B00004,C00018,A00002,B00004,C00018,A00002,...,**,**,**,**,**,**,**,**,**,180.0
u24295,1.418815e+09,1.418815e+09,male,A00001,B00001,C00012,A00001,B00001,C00012,**,...,**,**,**,**,**,**,**,**,**,120.0


In [18]:
train_data['gender'].value_counts(normalize=True)*100

female    78.019048
male      21.980952
Name: gender, dtype: float64

In [19]:
# checking the number of levels for product categories and sub categories
for i in product_list1:
    print(i)
    print('number of product categories',len(train_data[i+'_category'].unique()))
    print('number of product sub categories:',len(train_data[i+'_sub_category'].unique()))
    print('number of sub sub categories',len(train_data[i+'_sub_sub_category'].unique()))
    #print('number of products',len(train_data[i+'_product']))
    print('--'*20)

Product_0
number of product categories 11
number of product sub categories: 82
number of sub sub categories 330
----------------------------------------
Product_1
number of product categories 12
number of product sub categories: 73
number of sub sub categories 257
----------------------------------------
Product_2
number of product categories 12
number of product sub categories: 59
number of sub sub categories 203
----------------------------------------
Product_3
number of product categories 12
number of product sub categories: 58
number of sub sub categories 165
----------------------------------------
Product_4
number of product categories 10
number of product sub categories: 44
number of sub sub categories 140
----------------------------------------
Product_5
number of product categories 11
number of product sub categories: 42
number of sub sub categories 127
----------------------------------------
Product_6
number of product categories 9
number of product sub categories: 36
numb

In [20]:
print(train_data.shape)
print(test_data.shape)

(10500, 112)
(4500, 111)


In [21]:
# separating numeric and categorical attributes
numattr=['startTime','endTime','duration']
catattr=train_data.columns.difference(numattr)

train_data[catattr]=train_data[catattr].astype('category')
catattr=catattr.drop('gender')
test_data[catattr]=test_data[catattr].astype('category')


In [22]:
train_data[numattr].dtypes

startTime    float64
endTime      float64
duration     float64
dtype: object

# Train-test split

In [107]:
from sklearn.model_selection import train_test_split
X,y=train_data.loc[:,train_data.columns!='gender'],train_data['gender']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=23)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)


(9450, 111) (9450,)
(1050, 111) (1050,)


In [24]:
num_transformer=Pipeline(steps=[
    ('scaler',StandardScaler())
])
cat_transformer=Pipeline(steps=[
    ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
preprocessor=ColumnTransformer(transformers=[
    ('num',num_transformer,numattr),
    ('cat',cat_transformer,catattr)
])

## LogisticRegression

In [25]:
%%time
pipe_lr=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',LogisticRegression(C=0.5,max_iter=1000,solver='sag',random_state=23,n_jobs=3))
])
pipe_lr.fit(X_train,y_train)


Wall time: 5.11 s


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['startTime', 'endTime',
                                                   'duration']),
                                                 

In [76]:
# predicting and model evaluation
pred_train_logreg=pipe_lr.predict(X_train)
pred_test_logreg=pipe_lr.predict(X_test)
forecast_logreg=pipe_lr.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train_logreg))
print('test_accuracy:',accuracy_score(y_test,pred_test_logreg))

train accuracy: 0.8804232804232804
test_accuracy: 0.8733333333333333


## Random Forest

In [27]:
pipe_rf=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',RandomForestClassifier(n_jobs=3))
])
Kf=KFold(n_splits=5,random_state=23)

param_grid={'classifier__n_estimators':[200,300,400],
            'classifier__max_depth':[40],
            'classifier__max_features':[0.3,0.5],
            'classifier__min_samples_leaf':[4,6,8,10],
            'classifier__criterion':['gini','entropy']
            }
#rf_grid=RandomizedSearchCV(pipe_rf,param_grid,cv=Kf)
rf_grid=GridSearchCV(pipe_rf,param_grid,cv=Kf,n_jobs=3)


In [45]:
%%time
rf_grid.fit(X_train,y_train)

Wall time: 44min 2s


GridSearchCV(cv=KFold(n_splits=5, random_state=23, shuffle=False),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('scaler',
                                                                                          StandardScaler(copy=True,
                                                                                                    

In [46]:
rf_grid.best_params_

{'classifier__criterion': 'entropy',
 'classifier__max_depth': 40,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 4,
 'classifier__n_estimators': 400}

In [47]:
# predicting and model evaluation
pred_train_rf=rf_grid.predict(X_train)
pred_test_rf=rf_grid.predict(X_test)
forecast_rf=rf_grid.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train_rf))
print('test_accuracy:',accuracy_score(y_test,pred_test_rf))

train accuracy: 0.8926984126984127
test_accuracy: 0.8809523809523809


In [75]:
#getting results directly from saved model
#model1=joblib.load('rfModel1.sav')
#pred_train_rf=model1.predict(X_train)
#pred_test_rf=model1.predict(X_test)
#forecast_rf=model1.predict(test_data)

## Support Vector machine

In [31]:
pipe_svc=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',SVC(C=.8,kernel='rbf'))
])
pipe_svc.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True))],
                                                           verbose=False),
                                                  ['startTime', 'endTime',
                                                   'duration']),
                                                 

In [82]:
# predicting and model evaluation
pred_train_svc=pipe_svc.predict(X_train)
pred_test_svc=pipe_svc.predict(X_test)
forecast_svc=pipe_svc.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train_svc))
print('test_accuracy:',accuracy_score(y_test,pred_test_svc))

train accuracy: 0.8697354497354497
test_accuracy: 0.8638095238095238


# Gradient bossting classifier

In [41]:
from sklearn.ensemble import GradientBoostingClassifier
pipe_gbm=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',GradientBoostingClassifier(random_state=23))
])

param_grid={'classifier__n_estimators':[150,200,250],
            'classifier__max_depth':[20,40,80],
            'classifier__max_features':[0.4,0.5,0.7],
            'classifier__subsample':[0.6,0.4]
            }
gbm_grid=RandomizedSearchCV(pipe_gbm,param_grid,cv=3,n_jobs=3,verbose=True)


In [42]:
%%time
gbm_grid.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  30 out of  30 | elapsed: 11.2min finished


Wall time: 13min 13s


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('num',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('scaler',
                                                                                                StandardScaler(copy=True,
                                                                                               

In [43]:
gbm_grid.best_params_

{'classifier__subsample': 0.4,
 'classifier__n_estimators': 250,
 'classifier__max_features': 0.5,
 'classifier__max_depth': 80}

In [77]:
pred_train_gbm=gbm_grid.predict(X_train)
pred_test_gbm=gbm_grid.predict(X_test)
forecast_gbm=gbm_grid.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train))
print('test_accuracy:',accuracy_score(y_test,pred_test_gbm))

train accuracy: 0.8697354497354497
test_accuracy: 0.8695238095238095


## Adaboost

In [51]:
from sklearn.ensemble import AdaBoostClassifier
adb=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',AdaBoostClassifier(random_state=23))
])
param_grid={
           'classifier__n_estimators':[50,100,200,400,800,1600]}

adb_grid=GridSearchCV(adb,param_grid,cv=3)

In [52]:
%%time
adb_grid.fit(X_train,y_train)

Wall time: 6min 20s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('scaler',
                                                                                          StandardScaler(copy=True,
                                                                                                         with_mean=True,
                                        

In [78]:
pred_train_adb=adb_grid.predict(X_train)
pred_test_adb=adb_grid.predict(X_test)
forecast_adb=adb_grid.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train))
print('test_accuracy:',accuracy_score(y_test,pred_test_adb))

train accuracy: 0.8697354497354497
test_accuracy: 0.8657142857142858


## XGBoost

In [56]:
# XGBoost
pipe_xgb=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('classifier',xgb.XGBClassifier(random_state=23))
])
param_grid={'classifier__max_depth':[20,40,80],
           'classifier__n_estimators':[50,100,200,400,800,1600]}

xgb_grid=GridSearchCV(pipe_xgb,param_grid,cv=3)

In [57]:
%%time
xgb_grid.fit(X_train,y_train)

Wall time: 1h 8min 1s


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('num',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('scaler',
                                                                                          StandardScaler(copy=True,
                                                                                                         with_mean=True,
                                        

In [80]:
pred_train_xgb=xgb_grid.predict(X_train)
pred_test_xgb=xgb_grid.predict(X_test)
forecast_xgb=xgb_grid.predict(test_data)
print('train accuracy:',accuracy_score(y_train,pred_train_xgb))
print('test_accuracy:',accuracy_score(y_test,pred_test_xgb))

train accuracy: 0.9263492063492064
test_accuracy: 0.8828571428571429


In [59]:
# saving trained model
#filename='xgbModel1.sav'
#joblib.dump(xgb_grid,filename)

['xgbModel1.sav']

In [61]:
#loading saved model
#model1=joblib.load('rfModel1.sav')
#pred_test_rf=model1.predict(X_test)
#forecast_rf=model1.predict(test_data)

In [60]:
'''test_data['gender']=forecast_xgb
df=test_data['gender']
df.to_csv('predictions.csv',index=True,header=True)'''